In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import plotly
import plotly.express as px



In [21]:

df = pd.read_csv(r"athlete_events.csv")


In [22]:
df = df.fillna({
    'Age': df['Age'].median(),
    'Height': df['Height'].median(),
    'Weight': df['Weight'].median()
})

df = df.astype({'Age': 'uint8', 'Height': 'uint8', 'Weight': 'uint8', 'ID': 'uint32' , 'Year': 'int16'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      271116 non-null  uint32
 1   Name    271116 non-null  object
 2   Sex     271116 non-null  object
 3   Age     271116 non-null  uint8 
 4   Height  271116 non-null  uint8 
 5   Weight  271116 non-null  uint8 
 6   Team    271116 non-null  object
 7   NOC     271116 non-null  object
 8   Games   271116 non-null  object
 9   Year    271116 non-null  int16 
 10  Season  271116 non-null  object
 11  City    271116 non-null  object
 12  Sport   271116 non-null  object
 13  Event   271116 non-null  object
 14  Medal   39783 non-null   object
dtypes: int16(1), object(10), uint32(1), uint8(3)
memory usage: 23.0+ MB


In [23]:
print(df['Age'].median())
print(df['Age'].mean())


24.0
25.502493397660043


# ITALIA

In [24]:
# Anonymisera namn med SHA-256
df["Name"] = df["Name"].apply(
    lambda x: hashlib.sha256(str(x).encode()).hexdigest()
)

# Fyll tomma medaljer med "None" = ingen medalj
df["Medal"] = df["Medal"].fillna("None")


italydf_anon = df[df["NOC"] == "ITA"].copy()

italydf_anon.head()


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
158,62,1c1026605c0607937a724d3d839d964064f6907b3b2e0e...,M,21,198,90,Italy,ITA,2016 Summer,2016,Summer,Rio de Janeiro,Rowing,Rowing Men's Coxless Pairs,Bronze
197,91,d5ce4ad5e9aa7549d6069b66cfefa8df14d801fd845cc7...,M,27,190,80,Italy,ITA,2012 Summer,2012,Summer,London,Athletics,Athletics Men's 110 metres Hurdles,None
198,92,207f8f997895f35185acbd1e26fab5176d8b3dc6f3300f...,M,21,180,73,Italy,ITA,2008 Summer,2008,Summer,Beijing,Football,Football Men's Football,None
214,103,5a52f67df03607d3096242e2779913d915ccb30d26438f...,M,25,175,70,Italy,ITA,1936 Summer,1936,Summer,Berlin,Modern Pentathlon,Modern Pentathlon Men's Individual,Bronze
218,106,e54ebcdcbc9a487cea464c1def380df0dc1ea3d7f4ba83...,M,22,188,96,Italy,ITA,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Men's Quadruple Sculls,Gold


In [25]:
# Top 10 sporter där Italien tagit flest medaljer

# En meddalj per lag, för att få korrekt OS statistik 
ita_medals_unique = (
    italydf_anon[italydf_anon["Medal"] != "None"]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)

medals_by_sport = (
    ita_medals_unique
    .groupby("Sport")["Medal"]
    .count()
    .sort_values(ascending=False)
)

fig_medal_sport = px.bar(
    medals_by_sport.head(10),
    title="Top 10 sporter där Italien tagit flest medaljer",
    labels={"index": "Sport", "value": "Antal medaljer"}
)
fig_medal_sport.show()



In [26]:
# --- Sommar-OS ---
ita_summer_unique = (
    italydf_anon[
        (italydf_anon["Medal"] != "None") &
        (italydf_anon["Season"] == "Summer")
    ]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)
ita_summer_unique["Year"] = ita_summer_unique["Games"].str[:4].astype(int)

summer = (
    ita_summer_unique
    .groupby("Year")["Medal"]
    .count()
    .reset_index()
)
summer["Season"] = "Sommar"

# --- Vinter-OS ---
ita_winter_unique = (
    italydf_anon[
        (italydf_anon["Medal"] != "None") &
        (italydf_anon["Season"] == "Winter")
    ]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)
ita_winter_unique["Year"] = ita_winter_unique["Games"].str[:4].astype(int)

winter = (
    ita_winter_unique
    .groupby("Year")["Medal"]
    .count()
    .reset_index()
)
winter["Season"] = "Vinter"

combined = pd.concat([summer, winter], ignore_index=True)

fig_medal_year = px.line(
    combined,
    x="Year",
    y="Medal",
    color="Season",
    markers=True,
    title="Medaljer per år - Italien (Sommar och vinter)"
)

fig_medal_year.update_layout(
    legend_title_text="Säsong",
    yaxis_title="Antal medaljer"
)

fig_medal_year.show()


In [27]:
# Sommar-OS
ita_summer_unique = (
    ita[
        (ita["Medal"] != "None") &
        (ita["Season"] == "Summer")
    ]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)

medals_by_games_summer = (
    ita_summer_unique
    .groupby("Games")["Medal"]
    .count()
    .reset_index()
)

fig = px.line(
    medals_by_games_summer,
    x="Games",
    y="Medal",
    title="Medaljer per OS - Italien Sommar",
    markers=True
)
fig.show()

# Vinter-OS
ita_winter_unique = (
    ita[
        (ita["Medal"] != "None") &
        (ita["Season"] == "Winter")
    ]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)

medals_by_games_winter = (
    ita_winter_unique
    .groupby("Games")["Medal"]
    .count()
    .reset_index()
)

fig = px.line(
    medals_by_games_winter,
    x="Games",
    y="Medal",
    title="Medaljer per OS - Italien Vinter",
    markers=True
)
fig.show()  


NameError: name 'ita' is not defined

In [ ]:
# Histogram över åldrar för italienska 

fig = px.histogram(
    italydf_anon,
    x="Age",
    nbins=30,                     
    title="Åldersfördelning - Italienska OS-idrottare",
    labels={"Age": "Ålder"},
)

fig.show()


In [ ]:
# Antal deltagare per OS – Sommar 
ita_summer_participants = (
    ita[ita["Season"] == "Summer"]
    .drop_duplicates(subset=["Games", "ID"])     # en deltagare per OS
    .groupby("Games")["ID"]
    .count()
    .reset_index()
)

fig = px.line(
    ita_summer_participants,
    x="Games",
    y="ID",
    title="Antal italienska deltagare per OS - Sommar",
    markers=True,
    labels={"ID": "Antal deltagare"}
)
fig.show()


ita_winter_participants = (
    ita[ita["Season"] == "Winter"]
    .drop_duplicates(subset=["Games", "ID"])     # en deltagare per OS
    .groupby("Games")["ID"]
    .count()
    .reset_index()
)

fig = px.line(
    ita_winter_participants,
    x="Games",
    y="ID",
    title="Antal italienska deltagare per OS - Vinter",
    markers=True,
    labels={"ID": "Antal deltagare"}
)
fig.show()



# Fencing

In [28]:
fencing = italydf_anon[italydf_anon["Sport"] == "Fencing"].copy()

fencing_unique_medals = (
    fencing[fencing["Medal"] != "None"]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)


In [ ]:
medals_by_type_fen = (
    fencing_unique_medals
    .pivot_table(
        index="Year",
        columns="Medal",
        values="ID",      # bara för count
        aggfunc="count",
        fill_value=0
    )
    .reset_index()
)
# Beräkna totala medaljer per år
medals_type_cols = [c for c in medals_by_type_fen.columns if c in ["Gold", "Silver", "Bronze"]]
medals_by_type_fen["Total"] = medals_by_type_fen[medals_type_cols].sum(axis=1)

fig_fenc_medal = px.bar(
    medals_by_type_fen,
    x="Year",
    y="Total",
    title="Italy Fencing - Total Medals per Year",
    labels={"Total": "Total Medals"}
)
fig_fenc_medal.show()


In [30]:
medals_by_type = (
    fencing_unique_medals
    .pivot_table(
        index="Year",
        columns="Medal",
        values="ID",      # bara för count
        aggfunc="count",
        fill_value=0
    )
    .reset_index()
)

fig1 = px.bar(
    medals_by_type,
    x="Year",
    y=["Gold", "Silver", "Bronze"],
    title="Italy Fencing - Medals per Year",
    labels={"value": "Number of Medals", "variable": "Medal Type"},
    color_discrete_map={
        "Gold": "#F6D411",
        "Silver": "#D7D4D4",
        "Bronze": "#CD7532"
    }
)
fig1.update_layout(barmode="stack")
fig1.show()


In [32]:
medals_by_type = (
    fencing_unique_medals
    .pivot_table(
        index="Year",
        columns="Medal",
        values="ID",      # bara för count
        aggfunc="count",
        fill_value=0
    )
    .reset_index()
)

fig1 = px.bar(
    medals_by_type,
    x="Year",
    y=["Gold", "Silver", "Bronze"],
    title="Italy Fencing - Medals per Year",
    labels={"value": "Number of Medals", "variable": "Medal Type"},
    color_discrete_map={
    "Gold": "#F6D411",
    "Silver": "#D7D4D4",
    "Bronze": "#CD7532"
}
)
fig1.update_layout(barmode="stack")
fig1.show()

# Beräkna totala medaljer per år
medals_type_cols = [c for c in medals_by_type.columns if c in ["Gold", "Silver", "Bronze"]]
medals_by_type["Total"] = medals_by_type[medals_type_cols].sum(axis=1)

fig2 = px.bar(
    medals_by_type,
    x="Year",
    y="Total",
    #markers=True,
    title="Italy Fencing - Total Medals per Year",
    labels={"Total": "Total Medals"}
)
fig2.show()


1900–1920 – Uppbyggnadsperiod
Italien har en lång fäktningstradition, men sporten är ännu inte centraliserad nationellt. Träningen sker i lokala fäktskolor och militära akademier, och framgångarna beror främst på individuella mästare snarare än på ett enhetligt system.

1920–1936 – Den stora guldåldern
Under denna period exploderar Italiens OS-framgångar i fäktning. Fascistregimen använder idrott som propaganda och investerar tungt i elitverksamhet. Fäktning prioriteras särskilt eftersom sporten symboliserar disciplin, tradition, kontroll och militär precision, ideal som regimen vill förknippas med.
På 1920- och 30-talet byggs specialiserade träningscenter, fäktning integreras i militär och polis, och italienska tränare utvecklar moderna tekniker som blir internationell standard. Resultatet är att Italien blir världsledande inom framför allt florett och sabel, med flera OS som domineras av italienska fäktare.

1948–1960 – Efterkrigsåterhämtning
Efter andra världskriget faller regimen, men fäktningens struktur finns kvar. Italien har fortfarande ett av världens bästa träningssystem, och nya talangfulla generationer växer fram. Fäktningen stabiliseras på en hög nivå, med återkommande OS-medaljer och flera internationella stjärnor.

1960–1980 – Dipp under “Blyåren” och hård global konkurrens
Under gli anni di piombo  “Blyåren” drabbas Italien av terrorism, politisk extremism, ekonomisk instabilitet och social oro. Idrotten tappar resurser, särskilt individuella tekniska sporter som fäktning. Samtidigt ökar konkurrensen kraftigt: Sovjet, Ungern och andra öststater driver extremt välfinansierade elitprogram.
Italien går dessutom igenom en generationsväxling och saknar tillfälligt samma bredd som under storhetstiden. Tillsammans leder detta till en markant nedgång i OS-resultaten.

1980–Nutid – Modernisering och ny uppbyggnad
Från 1980-talet börjar Italien reformera sina fäktningsförbund och modernisera träningsmetoderna. Nya fäktskolor, förbättrade ungdomsprogram och mer professionell fysisk träning återuppbygger sportens nivå. Under slutet av 80-talet och 90-talet återvänder Italien till medaljtoppen i flera discipliner.

In [ ]:
# Fäktning för alla länder
fencing_all = df[df["Sport"] == "Fencing"].copy()

fencing_all_unique_medals = (
    fencing_all[fencing_all["Medal"] != "None"]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)

medals_country = (
    fencing_all_unique_medals
    .groupby("NOC")["Medal"]
    .count()
    .reset_index()
    .sort_values("Medal", ascending=False)
)

px.bar(
    medals_country.head(20),
    x="NOC",
    y="Medal",
    title="Fencing  Medal Distribution by Country",
    labels={"NOC": "Country", "Medal": "Number of Medals"}
).show()


Italien ligger bland de allra främsta nationerna tillsammans med  Frankrike, Ungern och Ryssland, vilket speglar ländernas långa fäktningstraditioner och välutvecklade tekniska skolor. Grafen visar tydligt hur fäktningen är starkt centrerad till några få, historiskt etablerade fäktningsnationer

In [ ]:

fencing = ita[ita["Sport"] == "Fencing"].copy()
fencing["Group"] = "Fencing"

other_sports = ita[ita["Sport"] != "Fencing"].copy()
other_sports["Group"] = "Other sports"

age_compare = pd.concat([fencing, other_sports], ignore_index=True)

# Medelålder per grupp
mean_age = (
    age_compare.groupby("Group")["Age"]
    .mean()
    .reset_index()
    .round(1)
)

fig = px.histogram(
    age_compare,
    x="Age",
    nbins=30,
    histnorm="percent",
    facet_row="Group",
    title="Age Distribution - Fencing vs Other Italian Sports",
    labels={"Age": "Age", "Group": "Group"}
)
fig.update_layout(
    height=700,
    margin=dict(t=80, b=40),
    font=dict(size=14),
)

# Adjust axes to remove the messy right-end overflow
fig.update_xaxes(range=[10, 50])     # adjust if you have older athletes
fig.update_yaxes(matches=None)
fig.show()


fäktare har en större andel aktiva i högre åldrar jämfört med andra sporter. Det speglar att fäktning är en tekniskt krävande sport där erfarenhet, precision och taktiskt kunnande gör att många når sin topp senare i karriären.

In [ ]:
# Medaljer per eent
medals_by_event = (
    fencing_unique_medals
    .groupby("Event")["Medal"]
    .count()
    .sort_values(ascending=False)  
    .reset_index()
)

fig = px.bar(
    medals_by_event,
    x="Event",
    y="Medal",
    title="Italy Fencing - Medals per Event",
    labels={"Medal": "Number of Medals", "Event": "Event"}
)

fig.update_xaxes(tickangle=45)
fig.show()
